## Téléchargements des modules et des fichiers

Pour le nettoyage des données, on utilise trois librairies dont deux spécifiques aux Data Sciences : Pandas et NumPy. On importe aussi la librairie datetime pour modifier les dates en un objet spécifique 'Date'.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

Une fois les différents modules importés, on charge les fichiers bruts nécessaires à notre analyse. On dispose d'un jeu de données composé de trois fichiers : un fichier contenant les ventes, nommé transactions, mais qui sera renommer sells ; un fichier contenant les informations concernant les clients nommé customers puis renommer custom et un fichier contenant les informations les produits vendus se nommant products et qui sera renommer prod pour le reste de l'analyse.

In [2]:
sells = pd.read_csv('data/transactions.csv')
custom = pd.read_csv('data/customers.csv')
prod = pd.read_csv('data/products.csv')

## Nettoyage des données des produits

Le nettoyage et l'analyse des données manquantes aberrantes se feront d'abord fichier par fichier. Une fois cette étape terminée, le fichier sera télécharger sous un nouveau nom pour ne pas modifier l'intégrité du fichier source. 

Pour se familiariser avec les données, on affiche les premières lignes du DataFrame prod. 

In [3]:
print(prod.head())

  id_prod  price  categ
0  0_1421  19.99      0
1  0_1368   5.13      0
2   0_731  17.99      0
3   1_587   4.99      1
4  0_1507   3.99      0


Ce dataframe est composé de trois colonnes contenant les identifiants des produits, les prix des produits et la catégorie d'appartenance de chacun des produits.

### Analyse des types dans le Dataframe

On regarde le détail des types de données contenues dans le dataframe.

In [4]:
prod.dtypes

id_prod     object
price      float64
categ        int64
dtype: object

Il serait plus intéressant de modifier le type de la colonne 'categ' en une valeur catégorielle. 

In [5]:
prod['categ'] = prod['categ'].astype('category')
prod.dtypes

id_prod      object
price       float64
categ      category
dtype: object

In [6]:
print(prod)

     id_prod   price categ
0     0_1421   19.99     0
1     0_1368    5.13     0
2      0_731   17.99     0
3      1_587    4.99     1
4     0_1507    3.99     0
...      ...     ...   ...
3282    2_23  115.99     2
3283   0_146   17.14     0
3284   0_802   11.22     0
3285   1_140   38.56     1
3286  0_1920   25.16     0

[3287 rows x 3 columns]


### Analyse des valeurs manquantes

Après la modification des types, on s'intéresse à la présence de données manquantes qui vont prendre la forme de NaN dans notre DataFrame. Pour identifier ces valeurs, on utiliser la fonction isna() associée à any().

In [7]:
prod.isna().any()

id_prod    False
price      False
categ      False
dtype: bool

Le DataFrame des produits ne contient aucune donnée manquante. 

### Analyse des valeurs dupliquées

Ensuite, on s'intéresse au valeur dupliquée. On crée une variable qui va contenir les valeurs dupliquées dans notre table pour pouvoir les isoler et voir la forme de ces valeurs dupliquées.

In [8]:
duplicated = prod.duplicated()
prod[duplicated]

,id_prod,price,categ


Le DataFrame prod ne présente pas de valeur dupliqué. On peut donc le laisser ainsi. 

### Autres analyses

Enfin, on s'intéresse à la présence de valeur aberrante. En utilisant la fonction .describe(), on observe la présence d'un prix égal à -1.00. On classe donc les prix par ordre croissant pour vérifier qu'il n'existe pas d'autres valeurs aberrante de ce type (produits valant 0€ par exemple).

In [9]:
prod.sort_values('price')

,id_prod,price,categ
731,T_0,-1.00,0
2355,0_202,0.62,0
2272,0_528,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0
...,...,...,...
2779,2_30,233.54,2
1436,2_167,236.99,2
394,2_158,247.22,2
724,2_76,254.44,2


Il n'existe qu'une seule valeur aberrante valant -1.00. De plus, on observe que l'id_prod associé n'a pas la même forme que les autres. On peut donc supprimer cette valeur. On transforme la valeur de l'id_prod associé à ce prix par un NaN pour enlever la ligne identifiée.

In [10]:
prod['id_prod'] = prod['id_prod'].replace('T_0', np.nan)
prod = prod.dropna()
prod.sort_values('price')

,id_prod,price,categ
2272,0_528,0.62,0
2355,0_202,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0
1530,0_1620,0.80,0
...,...,...,...
2779,2_30,233.54,2
1436,2_167,236.99,2
394,2_158,247.22,2
724,2_76,254.44,2


Une fois toutes les corrections faites, on exporte le nouveau dataframe sous forme de csv pour pouvoir l'utiliser pour l'analyse des ventes. Cela permet de conserver le fichier source intact si un retour en arrière est nécessaire. 

In [11]:
prod.to_csv('prod.csv')

## Nettoyage des données des clients

Concernant le fichier client, la méthode sera la même que pour les produits. On commence d'abord par afficher les premières lignes du DataFrame ainsi que les types pour voir la composition de celui-ci. 

In [12]:
print(custom.head())

  client_id sex  birth
0    c_4410   f   1967
1    c_7839   f   1975
2    c_1699   f   1984
3    c_5961   f   1962
4    c_5320   m   1943


Ce Dataframe est lui aussi composé de trois colonnes contenant les identifiants des clients, le sexe et l'année de naissance des clients. 

### Analyse des types

On s'intéresse ensuite aux types des données présents dans le fichier client.

In [13]:
custom.dtypes

client_id    object
sex          object
birth         int64
dtype: object

Le sex apparaît comme un objet, il peut être plus intéressant de changer ce type en variable catégorielle. 

In [14]:
custom['sex'] = custom['sex'].astype('category')
custom.dtypes

client_id      object
sex          category
birth           int64
dtype: object

### Analyse des données manquantes

On cherche la présence de valeurs manquantes (NaN) dans le fichier client. 

In [15]:
custom.isna().any()

client_id    False
sex          False
birth        False
dtype: bool

Le fichier client ne présente pas de données manquantes.

### Analyse des données dupliquées

On regarde la présence de données dupliquées et on les stocke dans une variable que l'on nomme duplicated_custom.

In [16]:
duplicated_custom = custom.duplicated()
custom[duplicated_custom]

,client_id,sex,birth


Ce DataFrame ne contient pas de données dupliquées. 

### Autres analyses

En ordonnant les valeurs, on observe la présence de deux client_id nommée sous une autre nomenclature. 

In [17]:
custom.sort_values('client_id')

,client_id,sex,birth
4299,c_1,m,1955
6894,c_10,m,1956
2137,c_100,m,1992
8472,c_1000,f,1966
3426,c_1001,m,1982
...,...,...,...
94,c_997,f,1994
2145,c_998,m,2001
7358,c_999,m,1964
2735,ct_0,f,2001


On observe la présence de deux clients avec des identifiants différents : ct_0 et ct_1. On remplace ces valeurs par NaN pour voir supprimer les lignes dans le DataFrame.

In [18]:
custom['client_id'] = custom['client_id'].replace('ct_0', np.nan)
custom['client_id'] = custom['client_id'].replace('ct_1', np.nan)
custom = custom.dropna()
custom.sort_values('client_id')

,client_id,sex,birth
4299,c_1,m,1955
6894,c_10,m,1956
2137,c_100,m,1992
8472,c_1000,f,1966
3426,c_1001,m,1982
...,...,...,...
7004,c_995,m,1955
2788,c_996,f,1970
94,c_997,f,1994
2145,c_998,m,2001


On enregistre le fichier sous le nom custom pour pouvoir l'utiliser dans l'analyse des ventes.

In [19]:
custom.to_csv('custom.csv')

## Nettoyage des données des ventes

On commence par afficher les premières lignes du DataFrame pour voir sa composition. 

In [20]:
print(sells.head())

  id_prod                        date session_id client_id
0  0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450
1   2_226  2022-02-03 01:55:53.276402   s_159142     c_277
2   1_374  2021-09-23 15:13:46.938559    s_94290    c_4270
3  0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597
4  0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242


Le DataFrame des ventes contient quatre colonnes : l'identifiant des produits, la date de la transaction, le numéro de la session et l'identifiant du client acheteur.

### Analyse des données manquantes

In [21]:
sells.isna().any()

id_prod       False
date          False
session_id    False
client_id     False
dtype: bool

Ce DataFrame ne présente pas de valeurs manquantes.

### Analyse des données dupliquées

On crée une variable contenant les ventes dupliquées.

In [22]:
duplicated_sells = sells.duplicated()
sells[duplicated_sells]

,id_prod,date,session_id,client_id
34387,T_0,test_2021-03-01 02:30:02.237443,s_0,ct_0
54813,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
57261,T_0,test_2021-03-01 02:30:02.237439,s_0,ct_1
58802,T_0,test_2021-03-01 02:30:02.237429,s_0,ct_0
60170,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_0
...,...,...,...,...
332594,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0
332705,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1
332730,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_1
333442,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1


On remarque la présence de données dupliquées dans notre DataFrame. Il s'agit de session test avec l'identifiant de session s_0. Ces sessions de tests expliquent la présence du produit avec l'identifiant 'T_0' et un prix de -1.00 dans le DataFrame des produit et la présence des clients ct_0 et ct_1 dans le fichier client. 

On supprime donc les valeurs dupliquées. 

In [23]:
sells = sells.drop_duplicates()
sells[duplicated_sells]

C:\Users\cecil\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,id_prod,date,session_id,client_id


Les valeurs dupliquées sont supprimées mais il reste encore des sessions s_0.

In [24]:
sells.loc[sells['session_id'] == 's_0']

,id_prod,date,session_id,client_id
1431,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1
2365,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_1
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1
5955,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0
7283,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_1
...,...,...,...,...
264317,T_0,test_2021-03-01 02:30:02.237416,s_0,ct_1
288918,T_0,test_2021-03-01 02:30:02.237415,s_0,ct_1
293107,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_0
298399,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1


Pour supprimer toutes les données issues des tests, on supprime toutes les sessions dont l'identifiant session vaut 's_0'. On remplace cette valeur par NaN pour supprimer les lignes concernés. 

In [25]:
sells = sells.replace('s_0', np.nan)
sells = sells.dropna()

### Analyse des types

On s'intéresse à l'analyse des types dans notre DataFrame débarrassé des sessions Test.

In [26]:
sells.dtypes

id_prod       object
date          object
session_id    object
client_id     object
dtype: object

Il peut être intéressant de présenter la date par un type date.

In [27]:
sells['date'] = pd.to_datetime(sells['date'])
print(sells.dtypes)

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object


On peut ainsi ranger les sessions par date.

In [28]:
sells = sells.sort_values('date')
sells = sells.set_index('date')
print(sells.head())

                           id_prod session_id client_id
date                                                   
2021-03-01 00:01:07.843138  0_1259        s_1     c_329
2021-03-01 00:02:26.047414  0_1390        s_2     c_664
2021-03-01 00:02:38.311413  0_1352        s_3     c_580
2021-03-01 00:04:54.559692  0_1458        s_4    c_7912
2021-03-01 00:05:18.801198  0_1358        s_5    c_2033


## Analyse et croisement avec les autres dataFrames

On cherche ensuite si des id_prod sont présents dans les ventes mais pas dans le fichier produit. Si c'est le cas, on supprime ce produit car il ne possède pas de prix. On fait de même avec les client_id.

In [29]:
sells['id_prod_prod'] = sells['id_prod'].isin(prod['id_prod'])
sells['client_id_custom'] = sells['client_id'].isin(custom['client_id'])
print(sells.head())

                           id_prod session_id client_id  id_prod_prod  \
date                                                                    
2021-03-01 00:01:07.843138  0_1259        s_1     c_329          True   
2021-03-01 00:02:26.047414  0_1390        s_2     c_664          True   
2021-03-01 00:02:38.311413  0_1352        s_3     c_580          True   
2021-03-01 00:04:54.559692  0_1458        s_4    c_7912          True   
2021-03-01 00:05:18.801198  0_1358        s_5    c_2033          True   

                            client_id_custom  
date                                          
2021-03-01 00:01:07.843138              True  
2021-03-01 00:02:26.047414              True  
2021-03-01 00:02:38.311413              True  
2021-03-01 00:04:54.559692              True  
2021-03-01 00:05:18.801198              True  


On crée deux colonnes contenant des booléens permettant de vérifier la présence d'identifiant différent. 

In [30]:
id_prod_false = sells[sells['id_prod_prod'] == False]
id_prod_false = id_prod_false.groupby('id_prod').mean()
print(id_prod_false)

         id_prod_prod  client_id_custom
id_prod                                
0_2245          False              True


On affiche les produits pour lesquels la variable id_prod_prod vaut False. Cela signifie que ce code produit se trouve uniquement dans les ventes mais pas dans les produits. On remarque que cela concerne uniquement un produit avec l'identifiant '0_2245'. Après vérification dans le DataFrame des produits, il n'existe pas de produit avec cet identifiant. On supprimera donc les sessions de vente ayant cet identifiant produit. 

On répète les étapes précédentes pour repérer la présence de client existants dans les sessions de vente mais qui ne sont pas existants dans le fichier.

In [31]:
id_client_false = sells[sells['client_id_custom'] == False]
id_client_false = id_client_false.groupby('client_id_custom').mean()
print(id_client_false)

Empty DataFrame
Columns: [id_prod_prod]
Index: []


Il n'y a pas de valeur 'False', on ne supprime pas de ligne pour cette condition. 

In [32]:
sells['id_prod'] = sells['id_prod'].replace('0_2245', np.nan)
sells = sells.dropna()

On supprime les deux colonnes crées pour le nettoyage avant d'enregistrer le DataFrame.

In [33]:
sells = sells.drop(columns=['client_id_custom', 'id_prod_prod'])

Une fois les corrections faites, on enregistre le fichier pour l'étape de l'analyse. 

In [34]:
sells.to_csv('sells.csv')